In [1]:
import os, cv2, glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Flatten, BatchNormalization, Convolution2D , MaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'cv2'

In [2]:
directory = 'C:/users/pvbha/Internship/PROJECT/model/'
imgtype = '/*.jpg'
path = os.path.dirname(directory)
folders = os.listdir(path)
folders

['train.csv',
 '.ipynb_checkpoints',
 'cactus.ipynb',
 'test',
 'train',
 'sample_submission.csv',
 'preprocess.ipynb']

In [3]:
folders.remove('.ipynb_checkpoints')
folders.remove('cactus.ipynb')
folders.remove('train.csv')
folders.remove('sample_submission.csv')
folders.remove('preprocess.ipynb')
folders

['test', 'train']

In [4]:
test = sorted([x for x in os.listdir(directory + folders[0] + '/') if x.endswith(imgtype[2:])])
train = sorted([x for x in os.listdir(directory + folders[1] + '/') if x.endswith(imgtype[2:])])
train

['0004be2cfeaba1c0361d39e2b000257b.jpg',
 '000c8a36845c0208e833c79c1bffedd1.jpg',
 '000d1e9a533f62e55c289303b072733d.jpg',
 '0011485b40695e9138e92d0b3fb55128.jpg',
 '0014d7a11e90b62848904c1418fc8cf2.jpg',
 '0017c3c18ddd57a2ea6f9848c79d83d2.jpg',
 '002134abf28af54575c18741b89dd2a4.jpg',
 '0024320f43bdd490562246435af4f90b.jpg',
 '002930423b9840e67e5a54afd4768a1e.jpg',
 '00351838ebf6dff6e53056e00a1e307c.jpg',
 '003519dd841a97ed16481fa0657df04d.jpg',
 '003bb64852016d9c87871ddd8e25ab03.jpg',
 '003ec9bcef67171ba49fe4c3b7c80aec.jpg',
 '003eeb9a86e36cd6328c778c15df890d.jpg',
 '0045d0f2aec739370eaefac79ee5b96c.jpg',
 '004fceec9b9b6a31dc9b0540fd69c692.jpg',
 '0051207eb794887c619341090de84b50.jpg',
 '0052d90950c3f08ed778d638a956fd43.jpg',
 '0057728c8522c4881af60c3105b6492e.jpg',
 '005aa32619d179665ecad3b227f8b537.jpg',
 '0062380830fe60c692a148afe64906ac.jpg',
 '00677f0440d465c2a685e33ded9bb729.jpg',
 '006bceec83605c63d844ed160cdbba89.jpg',
 '007a6a49d6049207f1716d1cc0fdf175.jpg',
 '007eba3edaf50d

In [5]:
df_train = pd.read_csv(directory + 'train.csv')
df_train.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [6]:
df_train['has_cactus'].value_counts()

1    13136
0     4364
Name: has_cactus, dtype: int64

In [7]:
y = df_train['has_cactus'].values.tolist()
y[:5]

[1, 1, 1, 1, 1]

In [8]:
def preprocess_image(image, n):
    
    img = cv2.imread(directory + folders[n] + '/' + image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh_bin = cv2.threshold(gray, 210, 255, cv2.THRESH_BINARY)
    return thresh_bin

In [9]:
X = []
for image in train:
    img = preprocess_image(image, 1)
    X.append(img)
    X = np.array(X).reshape(-1, 32, 32, 1)

In [10]:
train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
val_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [15]:
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 50

In [13]:
model = Sequential()
model.add(Convolution2D(128, (3, 3), input_shape = (32, 32, 1), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Convolution2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 128)       1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
__________

In [17]:
train_generator = train_datagen.flow(X_train, y_train, batch_size = batch_size)
validation_generator = val_datagen.flow(X_val, y_val, batch_size = batch_size)

In [ ]:
history = model.fit_generator(
    train_generator, 
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

Epoch 1/30
